In [1]:
#Dependencies
import pandas as pd
import numpy as np
import os
import sqlite3 

In [2]:
elec16_file = "Data/elec16v2.csv"
elec20_file = "Data/elec20v2.csv"
coord_file = "Data/us_county_latlng.csv"

coord_data = pd.read_csv(coord_file, low_memory=False)
elec20_data = pd.read_csv(elec20_file, low_memory=False)
elec16_data = pd.read_csv(elec16_file, low_memory=False)


In [3]:
coord_data.dtypes


fips_code      int64
name          object
lng          float64
lat          float64
dtype: object

In [4]:
elec20_data.dtypes


year              int64
office           object
party_simple     object
mode             object
votes             int64
countyname       object
countyfips        int64
candidatename    object
state_abbv       object
statefips         int64
combinedfips      int64
dtype: object

In [5]:
coord_data = coord_data.rename(columns = {"fips_code":"countyfips"})


In [6]:
coord_data


,countyfips,name,lng,lat
0,1059,Franklin,-87.843283,34.442381
1,13111,Fannin,-84.319296,34.864126
2,19109,Kossuth,-94.206898,43.204140
3,40115,Ottawa,-94.810589,36.835878
4,42115,Susquehanna,-75.800905,41.821277
...,...,...,...,...
3228,12029,Dixie,-83.158705,29.608068
3229,18017,Cass,-86.346207,40.761660
3230,26091,Lenawee,-84.066412,41.894694
3231,72003,Aguada,-67.175247,18.360392


In [7]:
map_data = pd.merge(elec20_data, coord_data, how="left", on=["countyfips"])
map_data

,year,office,party_simple,mode,votes,countyname,countyfips,candidatename,state_abbv,statefips,combinedfips,name,lng,lat
0,2020,US PRESIDENT,DEMOCRAT,EN DALECTIOY,239,NaN,0,JOSEPH R BIDEN,AK,2,200000,NaN,NaN,NaN
1,2020,US PRESIDENT,REPUBLICAN,ELECTION DAY,466,NaN,0,DONALD J TRUMP,AK,2,200000,NaN,NaN,NaN
2,2020,US PRESIDENT,DEMOCRAT,ELECTION DAY,47,NaN,0,JOSEPH R BIDEN,AK,2,200000,NaN,NaN,NaN
3,2020,US PRESIDENT,REPUBLICAN,ELECTION DAY,90,NaN,0,DONALD J TRUMP,AK,2,200000,NaN,NaN,NaN
4,2020,US PRESIDENT,DEMOCRAT,ELECTION DAY,85,NaN,0,JOSEPH R BIDEN,AK,2,200000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477319,2020,US PRESIDENT,REPUBLICAN,TOTAL,390,WESTON,56045,DONALD J TRUMP,WY,56,5656045,Weston,-104.567404,43.840315
477320,2020,US PRESIDENT,DEMOCRAT,TOTAL,67,WESTON,56045,JOSEPH R BIDEN,WY,56,5656045,Weston,-104.567404,43.840315
477321,2020,US PRESIDENT,REPUBLICAN,TOTAL,376,WESTON,56045,DONALD J TRUMP,WY,56,5656045,Weston,-104.567404,43.840315
477322,2020,US PRESIDENT,DEMOCRAT,TOTAL,63,WESTON,56045,JOSEPH R BIDEN,WY,56,5656045,Weston,-104.567404,43.840315


In [ ]:
#write the pandas dataframe to a sqlite table
final_map_data.columns = columns
final_map_data.to_sql(name,con,flavor='sqlite',schema=None,if_exists='replace',index=True,index_label=None, chunksize=None, dtype=None)

#connect to the database
conn = sqlite3.connect('database')
c = conn.curser()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE table RENAME TO old_table;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE new_table (col_1 TEXT PRIMARY KEY NOT NULL,
                            col_2 TEXT);

    INSERT INTO new_table SELECT * FROM old_table;

    DROP TABLE old_table;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
conn.close()

In [8]:
map_data_final = map_data.query("countyfips != 0")
map_data_final

,year,office,party_simple,mode,votes,countyname,countyfips,candidatename,state_abbv,statefips,combinedfips,name,lng,lat
1166,2020,US PRESIDENT,DEMOCRAT,TOTAL,141,AUTAUGA,1001,JOSEPH R BIDEN,AL,1,101001,Autauga,-86.642900,32.535142
1167,2020,US PRESIDENT,REPUBLICAN,TOTAL,226,AUTAUGA,1001,DONALD J TRUMP,AL,1,101001,Autauga,-86.642900,32.535142
1168,2020,US PRESIDENT,DEMOCRAT,TOTAL,315,AUTAUGA,1001,JOSEPH R BIDEN,AL,1,101001,Autauga,-86.642900,32.535142
1169,2020,US PRESIDENT,REPUBLICAN,TOTAL,1689,AUTAUGA,1001,DONALD J TRUMP,AL,1,101001,Autauga,-86.642900,32.535142
1170,2020,US PRESIDENT,DEMOCRAT,TOTAL,184,AUTAUGA,1001,JOSEPH R BIDEN,AL,1,101001,Autauga,-86.642900,32.535142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477319,2020,US PRESIDENT,REPUBLICAN,TOTAL,390,WESTON,56045,DONALD J TRUMP,WY,56,5656045,Weston,-104.567404,43.840315
477320,2020,US PRESIDENT,DEMOCRAT,TOTAL,67,WESTON,56045,JOSEPH R BIDEN,WY,56,5656045,Weston,-104.567404,43.840315
477321,2020,US PRESIDENT,REPUBLICAN,TOTAL,376,WESTON,56045,DONALD J TRUMP,WY,56,5656045,Weston,-104.567404,43.840315
477322,2020,US PRESIDENT,DEMOCRAT,TOTAL,63,WESTON,56045,JOSEPH R BIDEN,WY,56,5656045,Weston,-104.567404,43.840315


In [9]:
map_data_final.to_csv('Data/final_map_data.csv', index="False")